In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
trdf = pd.read_csv("data/train.csv")
tstdf = pd.read_csv("data/test.csv")
cctv_df = pd.read_csv("data/대구 CCTV 정보.csv",encoding='euc-kr')
child_df = pd.read_csv("data/대구 어린이 보호 구역 정보.csv",encoding='euc-kr')
light_df = pd.read_csv("data/대구 보안등 정보.csv",encoding='euc-kr')
park_df = pd.read_csv("data/대구 주차장 정보.csv",encoding='euc-kr')

tstdf.info(), cctv_df.info(), child_df.info(), light_df.info(), park_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   무인교통단속카메라관리번호  1065 non-null   object 
 1   시도명            1065 non-null   object 
 2   시군구명           1065 non-null   object 
 3   도로종류           1065 non-null   object 
 4   도로노선번호         288 non-null    object 
 5   도로노선명          1065 non-null   object 
 6   도로노선방향         1065 non-nu

C:\Users\user\AppData\Local\Temp\ipykernel_22848\1266655123.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  light_df = pd.read_csv("data/대구 보안등 정보.csv",encoding='euc-kr')


(None, None, None, None, None)

In [8]:
park_df.head()
park_df.drop('주차장관리번호',axis=1)

for c in park_df.columns:
    un = park_df[c].unique()
    print(f"칼럼 {c}의 유니크 값 {un}")
    print()    


# for c in trdf.columns:
#     unique_values = trdf[c].unique()
#     print(f"칼럼 '{c}'의 유니크한 값: {unique_values}")
#     print()

칼럼 주차장관리번호의 유니크 값 ['153-2-000001' '153-2-000002' '153-2-000003' '153-2-000004'
 '153-2-000005' '153-2-000006' '153-2-000007' '153-2-000008'
 '153-2-000009' '153-2-000010' '153-2-000011' '153-2-000012'
 '153-2-000013' '153-2-000014' '153-2-000015' '153-2-000016'
 '153-2-000017' '153-2-000018' '153-2-000019' '153-2-000020'
 '153-2-000021' '153-2-000022' '153-2-000023' '153-2-000024'
 '153-2-000025' '153-2-000026' '153-1-000027' '153-1-000028'
 '153-1-000029' '153-2-000030' '153-2-000031' '153-2-000032'
 '153-1-000033' '153-2-000034' '153-2-000035' '153-2-000036'
 '156-2-000001' '156-2-000003' '156-2-000004' '156-2-000005'
 '156-2-000006' '156-2-000007' '156-2-000008' '156-2-000009'
 '156-2-000010' '156-2-000011' '156-2-000012' '156-2-000013'
 '156-2-000014' '156-2-000015' '156-2-000016' '156-2-000017'
 '156-2-000018' '156-2-000019' '156-2-000020' '156-2-000021'
 '156-2-000022' '156-2-000023' '156-2-000024' '156-2-000025'
 '156-2-000026' '156-2-000027' '156-2-000028' '156-2-000029'
 '156-

In [5]:

# test에는 있지만 train에는 없는 칼럼 찾기
# columns_to_drop =  set(trdf.columns) - set(tstdf.columns) 

col = ['가해운전자 상해정도', '부상자수', '피해운전자 차종', '사망자수', '피해운전자 연령', '가해운전자 성별', '사고유형 - 세부분류', '피해운전자 성별', '가해운전자 차종', '중상자수', '경상자수', '피해운전자 상해정도', '가해운전자 연령', '법규위반']

# train에서 해당 칼럼 삭제
trdf = trdf.drop(columns=col, errors='ignore')

trdf = trdf.drop(columns=['ID' ,'사고일시'],axis=1)
tstdf = tstdf.drop(columns=['ID' , '사고일시'],axis=1)
trdf['시군구'] = trdf['시군구'].str.replace('대구광역시', '')
trdf[['구','동']] = trdf['시군구'].str.split(expand=True)
trdf = trdf.drop(columns=['시군구'], errors='ignore')
tstdf['시군구'] = tstdf['시군구'].str.replace('대구광역시', '')
tstdf[['구','동']] = tstdf['시군구'].str.split(expand=True)
tstdf = tstdf.drop(columns=['시군구'], errors='ignore')

# for c in trdf.columns:
#     unique_values = trdf[c].unique()
#     print(f"칼럼 '{c}'의 유니크한 값: {unique_values}")
#     print()

# for c in tstdf.columns:
#     unique_values = tstdf[c].unique()
#     print(f"칼럼 '{c}'의 유니크한 값: {unique_values}")
#     print()

In [6]:
# 트레이닝 데이터와 테스트 데이터의 주소 정보 결합
all_data = pd.concat([trdf[['동']], tstdf[['동']]])

# 원핫 인코딩
all_data_encoded = pd.get_dummies(all_data, columns=['동'],dtype=int)

# 트레이닝 데이터에 대한 원핫 인코딩 결과
df_train_encoded = all_data_encoded.iloc[:len(trdf)]
trdf = pd.concat([trdf, df_train_encoded], axis=1)

# 테스트 데이터에 대한 원핫 인코딩 결과
df_test_encoded = all_data_encoded.iloc[len(tstdf):]
tstdf = pd.concat([tstdf, df_test_encoded], axis=1)



# 트레이닝 데이터와 테스트 데이터의 '기상상태'와 '도로형태' 정보 결합
all_weather_road_data = pd.concat([trdf[['기상상태', '도로형태']], tstdf[['기상상태', '도로형태']]])

# 각 칼럼에 대한 원핫 인코딩
all_weather_road_data_encoded = pd.get_dummies(all_weather_road_data, columns=['기상상태', '도로형태'],dtype=int)

# 각 데이터셋에 인코딩 결과 합치기
trdf = pd.concat([trdf, all_weather_road_data_encoded.iloc[:len(trdf)]], axis=1)
tstdf = pd.concat([tstdf, all_weather_road_data_encoded.iloc[len(trdf):]], axis=1)


# object_columns = trdf.select_dtypes(include=['object']).columns
# object_columns = tstdf.select_dtypes(include=['object']).columns
# # 해당 열들에 대해서 원핫 인코딩 적용
# trdf = pd.get_dummies(trdf, columns=object_columns,dtype=int)
# tstdf = pd.get_dummies(tstdf, columns=object_columns,dtype=int)


trdf.info(), tstdf.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Columns: 221 entries, 요일 to 도로형태_주차장 - 주차장
dtypes: int32(213), int64(1), object(7)
memory usage: 34.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Columns: 220 entries, 요일 to 도로형태_주차장 - 주차장
dtypes: int32(213), object(7)
memory usage: 34.3+ MB


(None, None)

In [5]:
from sklearn.preprocessing import OneHotEncoder

trdf_eclo = trdf['ECLO']
# 트레이닝 데이터와 테스트 데이터의 object 열을 선택
object_columns_train = trdf.select_dtypes(include='object').columns
object_columns_test = tstdf.select_dtypes(include='object').columns

# 트레이닝 데이터와 테스트 데이터의 object 열을 합친 후 원핫 인코딩
combined_df = pd.concat([trdf[object_columns_train], tstdf[object_columns_test]])
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first'는 더미 변수의 중복을 피하기 위해 첫 번째 카테고리를 제거합니다.

# fit_transform을 사용하면 훈련 데이터에서 학습하고, 같은 변환을 테스트 데이터에 적용할 수 있습니다.
combined_encoded = encoder.fit_transform(combined_df)

# 변환된 값을 다시 트레이닝 데이터와 테스트 데이터로 나누기
trdf_encoded = pd.DataFrame(combined_encoded[:len(trdf)], columns=encoder.get_feature_names_out(object_columns_train))
tstdf_encoded = pd.DataFrame(combined_encoded[len(trdf):], columns=encoder.get_feature_names_out(object_columns_test))
trdf_encoded['ECLO'] = trdf_eclo

c:\Users\user\miniconda3\envs\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# 결과 확인
print("Encoded Training Data:")
print(trdf_encoded.info())
print("\nEncoded Test Data:")
print(tstdf_encoded.info())

Encoded Training Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Columns: 238 entries, 요일_목요일 to ECLO
dtypes: float64(237), int64(1)
memory usage: 71.9 MB
None

Encoded Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Columns: 237 entries, 요일_목요일 to 동_nan
dtypes: float64(237)
memory usage: 71.6 MB
None


In [7]:
from get import get,prepro
from factory_model import factory_model, create_models , models_cv
from cv import CV
from sklearn.metrics import accuracy_score
import torch
import numpy as np

In [8]:


# 훈련 데이터와 테스트 데이터를 numpy 배열로 변환
X_train = trdf_encoded.drop(columns=['ECLO']).values
y_train = trdf_encoded['ECLO'].values
X_test = tstdf_encoded.values

# numpy 배열을 PyTorch Tensor로 변환
X_trn = torch.tensor(X_train, dtype=torch.float32)
y_trn = torch.tensor(y_train, dtype=torch.float32)
X_tst = torch.tensor(X_test, dtype=torch.float32)

# 텐서 차원 확인
print("X_train_tensor shape:", X_trn.shape)
print("y_train_tensor shape:", y_trn.shape)
print("X_test_tensor shape:", X_tst.shape)


X_train_tensor shape: torch.Size([39609, 237])
y_train_tensor shape: torch.Size([39609])
X_test_tensor shape: torch.Size([39609, 237])


In [10]:
model = factory_model(X_trn,y_trn,"config.json")
model.fit(X_trn,y_trn)


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100, Average Training Loss: 11.7541
Epoch 2/100, Average Training Loss: 9.9883
Epoch 3/100, Average Training Loss: 10.0211
Epoch 4/100, Average Training Loss: 10.0363
Epoch 5/100, Average Training Loss: 9.9295
Epoch 6/100, Average Training Loss: 9.7826
Epoch 7/100, Average Training Loss: 9.7309
Epoch 8/100, Average Training Loss: 9.5892
Epoch 9/100, Average Training Loss: 9.4881
Epoch 10/100, Average Training Loss: 9.3416
Epoch 11/100, Average Training Loss: 9.2650
Epoch 12/100, Average Training Loss: 9.1617
Epoch 13/100, Average Training Loss: 9.0576
Epoch 14/100, Average Training Loss: 8.9901
Epoch 15/100, Average Training Loss: 8.9011
Epoch 16/100, Average Training Loss: 8.8167
Epoch 17/100, Average Training Loss: 8.7451
Epoch 18/100, Average Training Loss: 8.6624
Epoch 19/100, Average Training Loss: 8.6404
Epoch 20/100, Average Training Loss: 8.6078
Epoch 21/100, Average Training Loss: 8.5517
Epoch 22/100, Average Training Loss: 8.4961
Epoch 23/100, Average Training Loss: 8

KeyboardInterrupt: 